In [ ]:
!pip install --upgrade transformers accelerate datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 10.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
  Attempting uninstall: transformers
    Found existing installation: transformers 5.0.0
    Uninstalling transformers-5.0.0:
      Successfully uninstalled transformers-5.0.0


In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, cohen_kappa_score, mean_absolute_error, mean_squared_error
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
from datasets import Dataset
from transformers import EarlyStoppingCallback

In [ ]:
MODEL_NAME = "xlm-roberta-base"
N_FOLDS = 5
MAX_LEN = 512
BATCH_SIZE = 8
EPOCHS = 6
NUM_LABELS = 1

In [ ]:
df = pd.read_csv('training_data.csv')

In [ ]:

if df['labels'].min() == 1:
    print("Detected 1-indexed labels. Shifting to 0-indexed...")
    df['labels'] = df['labels'] - 1

df['labels'] = df['labels'].astype(float)

Detected 1-indexed labels. Shifting to 0-indexed...


In [ ]:
print("Unique labels found:", df['labels'].unique())
print("Min label:", df['labels'].min())
print("Max label:", df['labels'].max())

Unique labels found: [4. 3. 1. 5. 2. 6. 0. 7.]
Min label: 0.0
Max label: 7.0


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=MAX_LEN)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [ ]:
df['stratify_col'] = df['labels'].astype(int)
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)

fold_metrics = {'accuracy': [], 'qwk': [], 'mae': [], 'rmse': []}
best_score = -float('inf')
best_model_dir = "./best_essay_model"

print(f"Starting {N_FOLDS}-Fold CV (Regression Mode)...")

Starting 5-Fold CV (Regression Mode)...


In [ ]:
for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['stratify_col'])):
    print(f"\n{'='*20} FOLD {fold+1}/{N_FOLDS} {'='*20}")

    train_dataset = Dataset.from_pandas(df.iloc[train_idx])
    val_dataset = Dataset.from_pandas(df.iloc[val_idx])

    train_tokenized = train_dataset.map(tokenize_function, batched=True)
    val_tokenized = val_dataset.map(tokenize_function, batched=True)

    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=NUM_LABELS,
        problem_type="regression" # Explicitly force MSE Loss
    )

    
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        # Logits are shape (Batch, 1), flatten them
        predictions = logits.squeeze()

        # Round to nearest integer (1.2 -> 1, 4.8 -> 5)
        predictions_rounded = np.round(predictions)

        # Clip to valid range (0-7)
        predictions_final = np.clip(predictions_rounded, 0, 7)

        # Ensure labels are integers for metrics
        labels_int = labels.astype(int)
        preds_int = predictions_final.astype(int)

        acc = accuracy_score(labels_int, preds_int)
        qwk = cohen_kappa_score(labels_int, preds_int, weights='quadratic')
        mae = mean_absolute_error(labels_int, preds_int)
        rmse = np.sqrt(mean_squared_error(labels_int, preds_int))

        return {'accuracy': acc, 'qwk': qwk, 'mae': mae, 'rmse': rmse}

    training_args = TrainingArguments(
        output_dir=f'./results_reg_fold_{fold+1}',
        num_train_epochs=EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        learning_rate=2e-5,
        weight_decay=0.01,

        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="qwk",
        greater_is_better=True,
        save_total_limit=1,

        logging_steps=50,
        fp16=torch.cuda.is_available()
    )

    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_tokenized,
        eval_dataset=val_tokenized,
        compute_metrics=compute_metrics,

        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

 
    trainer.train()

   
    metrics = trainer.evaluate()
    print(f"Fold {fold+1}: QWK={metrics['eval_qwk']:.4f} | MAE={metrics['eval_mae']:.4f}")

    fold_metrics['accuracy'].append(metrics['eval_accuracy'])
    fold_metrics['qwk'].append(metrics['eval_qwk'])
    fold_metrics['mae'].append(metrics['eval_mae'])
    fold_metrics['rmse'].append(metrics['eval_rmse'])

    if metrics['eval_qwk'] > best_score:
        print(f"New Overall Best Score! Saving to {best_model_dir}...")
        best_score = metrics['eval_qwk']
        trainer.save_model(best_model_dir)
        tokenizer.save_pretrained(best_model_dir)

    del model, trainer
    torch.cuda.empty_cache()




for metric_name, values in fold_metrics.items():
    mean_val = np.mean(values)
    std_val = np.std(values)
    print(f"{metric_name.upper()}: {mean_val:.4f} ± {std_val:.4f}")


==================== FOLD 1/5 ====================


Map:   0%|          | 0/910 [00:00<?, ? examples/s]

Map:   0%|          | 0/228 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['roberta.embeddings.LayerNorm.weight', 'roberta.embeddings.LayerNorm.bias', 'roberta.encoder.layer.0.attention.output.LayerNorm.weight', 'roberta.encoder.layer.0.attention.output.LayerNorm.bias', 'roberta.encoder.layer.0.output.LayerNorm.weight', 'roberta.encoder.layer.0.output.LayerNorm.bias', 'roberta.encoder.layer.1.attention.output.LayerNorm.weight', 'roberta.encoder.layer.1.attention.output.LayerNorm.bias', 'roberta.encoder.layer.1.output.LayerNorm.weight', 'roberta.encoder.layer.1.output.LayerNorm.bias', 'roberta.encoder.layer.2.attention.output.LayerNorm.weight', 'roberta.encoder.layer.2.attention.output.LayerNorm.bias', 'roberta.encoder.layer.2.output.LayerNorm.weight', 'roberta.encoder.layer.2.output.LayerNorm.bias', 'roberta.encoder.layer.3.attention.output.LayerNorm.weight', 'roberta.encoder.layer.3.attention.output.LayerNorm.bias', 'roberta.encoder.layer.3.output.LayerNorm.weight', 'roberta.encoder.layer.3.output.Laye

Fold 5: QWK=0.8187 | MAE=0.6211
ACCURACY: 0.4780 ± 0.0236
QWK: 0.8317 ± 0.0170
MAE: 0.6046 ± 0.0394
RMSE: 0.8873 ± 0.0439


In [ ]:
print("\n" + "="*40)
print(f"FINAL REGRESSION RESULTS ({N_FOLDS}-Fold CV)")
print("="*40)

for metric_name, values in fold_metrics.items():
    print(f"{metric_name.upper()}: {np.mean(values):.4f} ± {np.std(values):.4f}")


FINAL REGRESSION RESULTS (5-Fold CV)
ACCURACY: 0.4780 ± 0.0236
QWK: 0.8317 ± 0.0170
MAE: 0.6046 ± 0.0394
RMSE: 0.8873 ± 0.0439
